In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

data = pd.read_csv("cleaned_input/train_consolidated.csv")

In [2]:
data.head()
data = data.drop(["msno", "Unnamed: 0"], axis=1)
data.shape

(504818, 124)

In [3]:
#data = pd.DataFrame([[1,2,3,5,6,7,8,9],[1,3,9,7,5,4,5,7]], columns=['a','b','c','d','e','f','g','is_churn'])

In [4]:
model_data = data.sample(frac=.1)
model_data.shape

(50482, 124)

In [5]:
X_train = model_data.drop(['is_churn'], axis=1);
y_train = model_data['is_churn'];

In [6]:
X_train.isnull().any()

trans_count                    False
date_min                       False
date_max                       False
date_count                     False
num_25_sum                     False
num_50_sum                     False
num_75_sum                     False
num_985_sum                    False
num_100_sum                    False
num_unq_sum                    False
total_secs_sum                 False
Unnamed: 0.1.1                  True
city                           False
bd                             False
registered_via                 False
registration_init_year         False
registration_init_month        False
registration_init_date         False
payment_method_id              False
payment_plan_days              False
plan_list_price                False
actual_amount_paid             False
is_auto_renew                  False
is_cancel                      False
discount                       False
is_discount                    False
amt_per_day                    False
m

In [7]:
print(np.all(np.isfinite(X_train)))
print(np.any(np.isnan(X_train)))

False
True


In [8]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train, scoring="neg_log_loss", train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [10]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_knn = {'n_neighbors':[1,3,6,10], 'weights':['uniform','distance']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}
#param_ann = {hidden_layer_sizes}
param_xgb = {
    'n_estimators':[100, 200, 300, 400, 500],
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2),
    'min_child_weight':[6,8,10,12],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
    }
param_xgb = {
    'n_estimators':[25, 50, 100, 200, 300, 400, 500],
    'learning_rate':[0.0001, 0.001, 0.01, 0.1]
    }
param_lgbm = {
    'learning_rate': [0.005, .05, .1, .5],
    'n_estimators': [8,16,24, 50, 100, 150],
    'num_leaves': [6,8,12,16, 32, 64],
    'max_depth': [-1, 10, 20, 50, 100, 200],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'colsample_bytree' : [0.6, 0.8, 0.9, 1],
    'subsample' : [0.5, 0.7,1],
    'reg_alpha' : [0, 0.5, 1, 1.2, 1.4],
    'reg_lambda' : [0, 0.5, 1 ,1.2, 1.4],
}

param_rfc = {'n_estimator':[5,10,15,20], 'max_features':[10,100,500,1000]}


In [15]:
model_list = {LogisticRegression():param_logit, KNeighborsClassifier():param_knn,
             SVC():param_svc, RandomForestClassifier():param_rfc};
model_list = {LogisticRegression():param_logit}
model_list = {XGBClassifier(n_jobs=3, verbose=True): param_xgb}
#model_list = {LGBMClassifier(nthread=3): param_lgbm}

In [16]:
for model,param in model_list.items():
    gscv = GridSearchCV(model, param);
    print("learning curve for\t{}".format(model))
    plot_learning_curve(gscv, X_train, y_train)

learning curve for	XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=3, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, verbose=True)


KeyboardInterrupt: 

In [ ]:
# Tune the parameters in a randomized fashion and control the distributions for colsample_bytree and subsample
from xgboost_tuner.tuner import tune_xgb_params

best_params, history = tune_xgb_params(
    cv_folds=3,
    label=y_train,
    metric_sklearn='neg_log_loss',
    metric_xgb='logloss',
    n_jobs=4,
    objective='binary:logistic',
    random_state=2017,
    strategy='randomized',
    train=X_train,
    colsample_bytree_loc=0.5,
    colsample_bytree_scale=0.2,
    subsample_loc=0.5,
    subsample_scale=0.2
)

[0]	train-logloss:0.606908+8.80946e-05	test-logloss:0.607269+0.000512625
[1]	train-logloss:0.536115+0.000152154	test-logloss:0.536634+0.000726114
[2]	train-logloss:0.477156+0.000140108	test-logloss:0.477912+0.000909318
[3]	train-logloss:0.427216+0.000184391	test-logloss:0.428228+0.00130051
[4]	train-logloss:0.384421+0.000113685	test-logloss:0.3856+0.00125145
[5]	train-logloss:0.347684+0.000100364	test-logloss:0.349005+0.00142289
[6]	train-logloss:0.316096+0.000107252	test-logloss:0.3177+0.00161207
[7]	train-logloss:0.28836+0.000367854	test-logloss:0.290132+0.00159999
[8]	train-logloss:0.264074+0.000508853	test-logloss:0.266011+0.00160042
[9]	train-logloss:0.242843+0.000665438	test-logloss:0.244978+0.00170367
[10]	train-logloss:0.224113+0.000856367	test-logloss:0.226475+0.00172524
[11]	train-logloss:0.207498+0.000935356	test-logloss:0.210076+0.00173122
[12]	train-logloss:0.192808+0.00103361	test-logloss:0.195566+0.00170456
[13]	train-logloss:0.179753+0.00108268	test-logloss:0.182752+0.0